# Quick intro

This is a jupyter notebook for testing / coding. So far, each code block is a separate test; unlike an ordinary notebook, they are not meant to run sequentially.

First, test to see jupyter is running correctly at all:


In [45]:
import os
import sys

print("hi!!!")

hi!!!


Next, running "toypyro". This started out as a copy of Fritz's code but I hope it evolves into a working version of ours.

In [2]:
from importlib import reload
import ei #import *
reload(ei)
from ei import *
import cProfile as profile

%prun result = trainGuide()

Yes, I will run.
Reloading polytopize.
ec:tensor([0.0000, 0.5000, 0.0000])
erc:tensor([[0., 1., 2.],
        [2., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]])
y[0]:tensor([[ 25., 171., 237.],
        [238.,  50.,  34.],
        [ 69.,  80.,  58.],
        [ 21.,  42.,  25.]])
svi.step(... 0 tensor([1.7596, 1.7940, 1.8125, 1.8220])


RuntimeError: Dimension out of range (expected to be in range of [-1, 0], but got 1)
      Trace Shapes:      
       Param Sites:      
        precinctpsi    18
         logsdrchat      
        logsdprchat      
              echat     3
             erchat  4  3
      Sample Sites:      
        all_ps dist     |
              value 17  |
param_residual dist     |
              value     |

In [ ]:
from importlib import reload
import ei_minbroken #import *
reload(ei_minbroken)
from ei_minbroken import *
import cProfile as profile

%prun result = trainGuide()

Next, run "polytopize", which has all of my pre-Fritz attempts to code.

In [29]:
import torch
ts = torch.tensor
import pyro.distributions as dist
import pyro
import cmult

t1 = ts([3.,4.])
t2 = ts([2.,5.])

m = t1.view(-1,1).mm(t1.view(1,-1))

print(m)

#means = torch.zeros(3,2)
#means[1,1] = 100.
with pyro.plate('dumb',3):
    #t3 = pyro.sample('irrelevant',dist.Normal(means,ts(1.)).to_event(1))
    t3 = pyro.sample('irrelevant',cmult.CMult(50,torch.ones(3,3,3)/3).to_event(2))

#print(t3)

mult = cmult.CMult(50,torch.ones(3)/3)
print(mult.log_prob(ts([0.,50,0])))
print(mult.log_prob(ts([0.,1,0])))

tensor([[ 9., 12.],
        [12., 16.]])
tensor(-54.9306)
tensor(-1.0986)


In [6]:
yhat1 = polytopize(4,3,pyro.get_param_store()["what_0"],
                 get_indep(4,3,ts([68.,20,25,56]),ts([63.,32,74])))

erchat = pyro.get_param_store()["erchat"]

print(pyro.get_param_store()["what_0"])
print(erchat)
print(recenter_rc(erchat))
print(pyro.get_param_store()["logsdrchat"])

tensor([[-0.3764, -0.7578],
        [-0.0823,  0.0420],
        [ 0.0860,  0.2053]], requires_grad=True)
tensor([[-0.0440, -0.0448,  0.1284],
        [-0.0490, -0.0334,  0.1120],
        [-0.0486, -0.0243,  0.0850],
        [-0.0567, -0.0134,  0.0715]], requires_grad=True)
tensor([[ 0.0031, -0.0173,  0.0341],
        [-0.0019, -0.0059,  0.0177],
        [-0.0015,  0.0032, -0.0093],
        [-0.0096,  0.0141, -0.0227]], grad_fn=<SubBackward0>)
tensor(-3.1610, requires_grad=True)


In [ ]:
import pstats
p = pstats.Stats(runstats)
p.strip_dirs().sort_stats(-1).print_stats()

In [4]:
import sys
import hessian

from importlib import reload
import polytopize #import *
reload(polytopize)
from polytopize import *

import tests


Yes, I will run.
Yes, I will run.
ge fail
loc tensor([[ 0.3147],
        [ 2.4440],
        [-4.0535],
        [ 2.2007]])
polytopedLoc tensor([[ 2.1063e-01,  2.1136e-02],
        [ 2.8514e-01, -1.8626e-09],
        [ 3.4959e-01,  9.9957e-02],
        [ 3.5462e-01,  1.1892e-02],
        [ 2.5462e+00,  3.2228e-01]])
ge fail
loc tensor([[-5.8855],
        [-5.2805],
        [ 5.4654],
        [ 0.1889]])
polytopedLoc tensor([[ 2.8885e-01,  1.1871e+00],
        [ 8.5629e-01,  2.0505e+00],
        [ 3.9984e-01, -1.4901e-08],
        [ 9.5542e-01,  1.5538e+00],
        [ 1.0063e+00,  9.9522e-01]])
Reloading cmult...
callable? <bound method TorchDistributionMixin.__call__ of Multinomial()>
callable? <bound method TorchDistributionMixin.__call__ of TorchCMult()>
Sampling multinomial: tensor([1., 2.])
Sampling cm2: tensor([0., 3.])
tensor(5.6022, grad_fn=<NegBackward>) tensor([[112.2500]])


# Test rank1torch (to get yhat from pi,n,v)

In [5]:
from importlib import reload
import rank1torch #import *
reload(rank1torch)
from rank1torch import *

test_solver()

Testing optimize_Q (50 tests): 
R=3, C=5, tolerance=0.001


AttributeError: module 'torch' has no attribute 'solve'

Finally, this is a cell for directly experimenting with pytorch and/or pyro. Basically, for getting the tensor syntax right. 

In [ ]:
import torch
import cmult
import pyro
from importlib import reload
reload(cmult)
from cmult import CMult

#print(CMult(probs=torch.tensor([1., 1., 1., 1.])).log_prob(torch.tensor([1.2, 1.5, 1., 1.])))

#print(CMult(100, torch.tensor([ 1., 1., 1., 1.])).sample())

R, C = (3,4)

y = pyro.distributions.Normal(0.,4.).sample(torch.Size([R-1,C-1]))


w = torch.cat((y,-y.sum(1).unsqueeze(1)),1)
w = torch.cat((w,-w.sum(0).unsqueeze(0)),0)

#print(w)
b = w.argmax()
print(b)
print(w,w[b//C,b % C])

tt = torch.zeros(R,C)
print(tt,R,C,tt[:(R-1),:(C-1)])

## Overview

Most SVI problems in pyro are coded as a model, a generic guide (such as: multivariate Gaussian in all parameters), and specific observations/data (passed as arguments to svi.step). For EI, that's going to be different; the observations are going to be built into the guide function, leaving nothing to include in the "data" argument to svi.step.

That means there is a lot of work for the guide to do. As usual, it must establish reasonable distributional families for the posterior of each of the hyperparameters. But for the latent parameters, the job of the guide is to take a "relative strength" number for each race/candidate/precinct combo, and turn that into a number of votes for each combo, such that those numbers obey all the constraints set by observations. This means that for each precinct (considered separately), the latent guide must:

-Find the "center point" where candidate preference is independent of race.

-Find the "basis vectors" (actually, there are more than enough of them to form a basis) which determine the directions to move in the space.

-For any given set of "relative strengths" which is a distance $d$ in a direction $\theta$, find the first constraint violated when moving in that direction, and the distance $r$ between the origin and that constraint.

-Project the "relative strengths" onto the numbers of votes, by moving $r(1-e^{-d})$ in direction $\theta$.

#### a = zs(2,2,2,2)
a[0,1,1,1] = 2
print(a[1,1])
print(a[0,1])
print(torch.max(a))
print(torch.distributions.exponential.Exponential(ts([1])).sample(4))

In [ ]:
print("hi","world")
a = zs(2,2,2,2)
a[0,1,1,1] = 2 
print(torch.max(a[0,1])) 
print(pyro.distributions.Exponential(1.).sample(torch.Size([3])))
a.add_(torch.ones(2,2,2))
print(torch.exp)